# Calculate Stats on Tuning Distributions

In [ ]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import scikit_posthocs as posthoc
from statsmodels.stats.multitest import multipletests
from hmmlearn import hmm

sys.path.insert(0, os.path.abspath(r'C:/Users/mmccann/repos/bonhoeffer/prey_capture/'))
import functions_plotting as fp

In [ ]:
def h5_to_dict(filepath):
    with pd.HDFStore(filepath, 'r') as store:
        keys = store.keys()
        data = {key[1:]: store[key] for key in keys}
    return data


def kruskal_w_dunn(data, nan_policy='omit', alpha=0.05):
    statistic, p_val = stats.kruskal(*data, nan_policy=nan_policy)
    dunn_test = posthoc.posthoc_dunn([*data], p_adjust='holm')

    # Return just the corrected pvals instead of a whole matrix
    if dunn_test.shape[-1] > 2:
        p_val_corrected = np.array([dunn_test.iloc[0, i] for i in range(1, dunn_test.shape[-1])] + [dunn_test.iloc[1, -1]])
    else:
        p_val_corrected = np.array([dunn_test.iloc[0, i] for i in range(1, dunn_test.shape[-1])])
    return p_val, p_val_corrected


def mannwhitneyu_w_bonferroni(data, nan_policy='omit', alpha=0.05):
    statistic, p_val = stats.mannwhitneyu(*data, nan_policy=nan_policy)
    reject, p_val_corrected, _, _ = multipletests(p_val, alpha=alpha, method='holm')
    return p_val, p_val_corrected


def wilcoxon_w_bonferroni(data, nan_policy='omit', alpha=0.05):
    statistic, p_val = stats.wilcoxon(*data, nan_policy=nan_policy)
    reject, p_val_corrected, _, _ = multipletests(p_val, alpha=alpha, method='holm')
    return p_val, p_val_corrected


def sample_df(data1, data2):
    df1 = data1.dropna()
    df2 = data2.dropna() 

    if len(df1) > len(df2):
        df1 = df1.sample(len(df2), replace=True)
    elif len(df2) > len(df1):
        df2 = df2.sample(len(df1), replace=True)
    else:
        pass

    return df1, df2


def bootstrap_tests(datasets, key, tuning_kind, method, n_iters=10000, alpha=0.05, bootstrap_all=False):
    data = [ds[key][tuning_kind].dropna() for ds in datasets]

    # First run the test on the raw data
    if 'wilcoxon' in method.__name__:
        raw_test = method(sample_df(*data), alpha=alpha)
    elif 'mannwhitneyu' in method.__name__:
        raw_test = method(data, alpha=alpha)
    else:
        raw_test = method(data, alpha=alpha)

    # Now run the bootstrapping
    p_vals = np.zeros(n_iters)
    p_vals_corrected = np.zeros((n_iters, len(data)))

    sample_size = len(max(data, key=len))

    # Run bootstrap
    for i in range(n_iters):

        if bootstrap_all:
            sample_data = [d.sample(sample_size, replace=True) for d in data]
            # sample_data1  = data2.sample(sample_size, replace=True)
        else:
            sample_data = [data[0]] + [d.sample(sample_size, replace=True) for d in data[1:]]
            
        p_val, p_val_corrected = method(sample_data, alpha=alpha)
        p_vals[i] = p_val
        p_vals_corrected[i, :] = p_val_corrected

    # 95% confidence interval of pvals
    ci = stats.t.interval(0.95, len(p_vals) - 1, loc=np.nanmean(p_vals), scale=stats.sem(p_vals, nan_policy='omit'))

    if method == kruskal_w_dunn:
        # Since this is for multiple groups, need to run this column-wise

        # Corrected 95% CIs
        ci_corrected = np.zeros((len(data), 2))
        for i in range(p_vals_corrected.shape[-1]):
            p_val_corr_col = p_vals_corrected[:, i]
            ci_col = stats.t.interval(0.95, len(p_val_corr_col) - 1, loc=np.nanmean(p_val_corr_col), scale=stats.sem(p_val_corr_col, nan_policy='omit'))
            ci_corrected[i, :] = ci_col
    else:
        ci_corrected = stats.t.interval(0.95, len(p_vals_corrected) - 1, loc=np.nanmean(p_vals_corrected), scale=stats.sem(p_vals_corrected, nan_policy='omit'))

    return raw_test, (p_vals, ci), (p_vals_corrected, ci_corrected)


def plot_bootstrap_distributions(data):
    # Plot the bootstrap distributions
    num_ds = len(data)
    cols = 3
    rows = num_ds//cols + 1
    fig, axes = plt.subplots(rows, cols, figsize=(5*cols, 5*rows))

    for i, (ds, ax) in enumerate(zip(data, axes.ravel())):

        raw_test = ds[0]
        bootstrap_tests = ds[1]
        corrected_bootstrap_tests = ds[2]

        ax.hist(bootstrap_tests[0], bins=20)
        ax.axvline(np.percentile(bootstrap_tests[0], 95), 0, 50, color='r')
        ax.axvline(bootstrap_tests[1][0], 0, 50, color='g')
        ax.axvline(bootstrap_tests[1][1], 0, 50, color='g')
        ax.axvline(raw_test[0], 0, 100, color='k')


    return fig, axes

In [ ]:
basedir = r"Z:\Prey_capture\WF_Figures"
still_or_all = 'full'
main_exp_path = os.path.join(basedir, still_or_all, r"multi_normal_ALL\stats.hdf5")
control_light_path  = os.path.join(basedir, still_or_all, r"control_normal_ALL\stats.hdf5")
control_dark_path = os.path.join(basedir, still_or_all, r"control_dark_ALL\stats.hdf5")
repeat_fixed_path = os.path.join(basedir, still_or_all, r"repeat_normal_VWheelWF\stats.hdf5")
repeat_free_path = os.path.join(basedir, still_or_all, r"repeat_normal_VTuningWF\stats.hdf5")
output_path = os.path.join(basedir, 'stats', f"{still_or_all}_tuning_stats.h5")

In [ ]:
main_exp_data = h5_to_dict(main_exp_path)
control_light_data = h5_to_dict(control_light_path)
control_dark_data = h5_to_dict(control_dark_path)
repeat_fixed_data = h5_to_dict(repeat_fixed_path)
repeat_free_data = h5_to_dict(repeat_free_path)

In [ ]:
method = kruskal_w_dunn
bootstrap_iters = 1000

In [ ]:
# main_exp_data['frac_vis_resp_fixed'].set_index(['mouse', 'day']).iloc[:10, :]

In [ ]:
# main_exp_data['frac_vis_resp_free'].count()

In [ ]:
# control_dark_data['frac_kinem_resp_fixed'].count()

In [ ]:
# control_light_data['frac_kinem_resp_fixed'].count()

# Kinematic Tuning

In [ ]:
def kinem_stats(data, ds_name, alpha=0.05):
    cols = data[0][ds_name].columns.to_list()

    stats = {}
    for col in cols:
        stats[col] = bootstrap_tests(data, ds_name, col,   
                                      kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)

    return stats

In [ ]:
# with pd.HDFStore(output_path, 'a') as store:
#     a = store['kruskal_kinem_tuning_free']

#     for var in a.index.to_list():
#         print(var)
#         print(a.loc[var, 'control_dark'][-1])

In [ ]:
# Fixed data
fixed_kinem = kinem_stats([main_exp_data, control_light_data, control_dark_data], 'frac_kinem_resp_fixed')
kinem_tuning_fixed = pd.DataFrame(fixed_kinem).T
kinem_tuning_fixed.columns = ['main', 'control_light', 'control_dark']

# Save it
with pd.HDFStore(output_path, 'a') as store:
    store[f'kruskal_kinem_tuning_fixed'] = kinem_tuning_fixed


# Free data
free_kinem = kinem_stats([main_exp_data, control_light_data, control_dark_data], 'frac_kinem_resp_free')
kinem_tuning_free = pd.DataFrame(free_kinem).T
kinem_tuning_free.columns = ['main', 'control_light', 'control_dark']

# Save it
with pd.HDFStore(output_path, 'a') as store:
    store[f'kruskal_kinem_tuning_free'] = kinem_tuning_free


# Visual Tuning

In [ ]:
vis_classes = ['', 'mod_', 'not_']

## Frac Visual Tuning - Fixed to Free

### Kruskal-Wallis

In [ ]:
for vis_class in vis_classes:
    # Do a Kruskal-Wallis test with posthoc Dunn test for multiple comparisons

    # Visual
    exp_fixed_free_vis = kruskal_w_dunn(sample_df(main_exp_data['frac_vis_resp_fixed'][f'{vis_class}visual'], 
                                                  main_exp_data['frac_vis_resp_free'][f'{vis_class}visual'])
                                                  )
    print('Fixed vs Free Visual:', exp_fixed_free_vis)


    # Orientation
    exp_fixed_free_ori = kruskal_w_dunn(sample_df(main_exp_data['frac_vis_resp_fixed'][f'{vis_class}orientation'], 
                                                  main_exp_data['frac_vis_resp_free'][f'{vis_class}orientation'])
                                                  )
    print('Fixed vs Free Orientation:', exp_fixed_free_ori)


    # Direction
    exp_fixed_free_dir = kruskal_w_dunn(sample_df(main_exp_data['frac_vis_resp_fixed'][f'{vis_class}direction'], 
                                                  main_exp_data['frac_vis_resp_free'][f'{vis_class}direction'])
                                                  )
    print('Fixed vs Free Direction:', exp_fixed_free_dir)

    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/kruskal_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
        store[f'{vis_class}vis/kruskal_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
        store[f'{vis_class}vis/kruskal_exp_fixed_free_dir'] = pd.DataFrame(exp_fixed_free_dir, index=['p_val', 'p_val_dunn_posthoc'])

### Wilcoxon Rank Sum

In [ ]:
# Do a Wilcox Rank-Sum test with Bonferroni correction

for vis_class in vis_classes:
    # Visual
    exp_fixed_free_vis = wilcoxon_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'][f'{vis_class}visual'], 
                                                        main_exp_data['frac_vis_resp_free'][f'{vis_class}visual']))
    print('Fixed vs Free Visual:', exp_fixed_free_vis)


    # Orientation
    exp_fixed_free_ori = wilcoxon_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'][f'{vis_class}orientation'], 
                                                        main_exp_data['frac_vis_resp_free'][f'{vis_class}orientation']))
    print('Fixed vs Free Orientation:', exp_fixed_free_ori)


    # Direction
    exp_fixed_free_dir = wilcoxon_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'][f'{vis_class}direction'], 
                                                        main_exp_data['frac_vis_resp_free'][f'{vis_class}direction']))
    print('Fixed vs Free Direction:', exp_fixed_free_dir)

    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/wilcoxon_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_bonferroni'])
        store[f'{vis_class}vis/wilcoxon_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_bonferroni'])
        store[f'{vis_class}vis/wilcoxon_exp_fixed_free_dir'] = pd.DataFrame(exp_fixed_free_dir, index=['p_val', 'p_val_bonferroni'])

### Mann-Whitney U

In [ ]:
# Do a Mann-Whitney U test with Bonferroni correction
for vis_class in vis_classes:
    # Visual
    exp_fixed_free_vis = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'][f'{vis_class}visual'], 
                                                main_exp_data['frac_vis_resp_free'][f'{vis_class}visual']))
    print('Fixed vs Free Visual:', exp_fixed_free_vis)

    # Orientation
    exp_fixed_free_ori = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'][f'{vis_class}orientation'], 
                                                main_exp_data['frac_vis_resp_free'][f'{vis_class}orientation']))
    print('Fixed vs Free Orientation:', exp_fixed_free_ori)

    # Direction
    exp_fixed_free_dir = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['frac_vis_resp_fixed'][f'{vis_class}direction'], 
                                                main_exp_data['frac_vis_resp_free'][f'{vis_class}direction']))
    print('Fixed vs Free Direction:', exp_fixed_free_dir)

    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_vis'] = pd.DataFrame(exp_fixed_free_vis, index=['p_val', 'p_val_bonferroni'])
        store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_ori'] = pd.DataFrame(exp_fixed_free_ori, index=['p_val', 'p_val_bonferroni'])
        store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_dir'] = pd.DataFrame(exp_fixed_free_dir, index=['p_val', 'p_val_bonferroni'])

## Frac Visual Tuning - Exp vs Controls

### Fixed Kruskal Wallis

In [ ]:
# Do a Kruskal-Wallis test with Dunn posthoc
for vis_class in vis_classes:
    # Visual
    frac_fixed_vis = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'frac_vis_resp_fixed', 'visual',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Fixed vs Controls Visual 95% CI', frac_fixed_vis[1][1])
    print('Fixed vs Controls Visual 95% CI - Corrected:', frac_fixed_vis[-1][1])
    print('Fixed vs Controls Visual Raw:', frac_fixed_vis[0])

    # Orientation
    frac_fixed_ori = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'frac_vis_resp_fixed', 'orientation',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Fixed vs Controls Ori 95% CI', frac_fixed_ori[1][1])
    print('Fixed vs Controls Ori 95% CI - Corrected:', frac_fixed_ori[-1][1])
    print('Fixed vs Controls Ori Raw:', frac_fixed_ori[0])

    # Direction
    frac_fixed_dir = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'frac_vis_resp_fixed', 'direction',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Fixed vs Controls Dir 95% CI', frac_fixed_dir[1][1])
    print('Fixed vs Controls Dir 95% CI - Corrected:', frac_fixed_dir[-1][1])
    print('Fixed vs Controls Dir Raw:', frac_fixed_dir[0])

    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/kruskal_frac_fixed_vis'] = pd.DataFrame(list(frac_fixed_vis), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
        store[f'{vis_class}vis/kruskal_frac_fixed_ori'] = pd.DataFrame(list(frac_fixed_ori), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
        store[f'{vis_class}vis/kruskal_frac_fixed_dir'] = pd.DataFrame(list(frac_fixed_dir), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T

In [ ]:
# with pd.HDFStore(output_path, 'a') as store:
#     a = store[f'{vis_class}vis/kruskal_frac_fixed_dir']
#     print(a.loc['ci', 'corrected_bootstrap_tests'])

In [ ]:
# frac_fixed_fig = plot_bootstrap_distributions([frac_fixed_gen, 
#                                                frac_fixed_vis, frac_fixed_vis_no_gen, 
#                                                frac_fixed_ori, frac_fixed_ori_no_gen,
#                                                frac_fixed_dir, frac_fixed_dir_no_gen])
# plt.show()

### Free Kruskal Wallis

In [ ]:
# Do a Kruskal-Wallis test with Dunn posthoc
for vis_class in vis_classes:
    # Visual
    frac_free_vis = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'frac_vis_resp_free', 'visual',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Free vs Controls Visual 95% CI', frac_free_vis[1][1])
    print('Free vs Controls Visual 95% CI - Corrected:', frac_free_vis[-1][1])
    print('Free vs Controls Visual Raw:', frac_free_vis[0])

    # Orientation
    frac_free_ori = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'frac_vis_resp_free', 'orientation',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Free vs Controls Ori 95% CI', frac_free_ori[1][1])
    print('Free vs Controls Ori 95% CI - Corrected:', frac_free_ori[-1][1])
    print('Free vs Controls Ori Raw:', frac_free_ori[0])

    # Direction
    frac_free_dir = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'frac_vis_resp_free', 'direction',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Free vs Controls Dir 95% CI', frac_free_dir[1][1])
    print('Free vs Controls Dir 95% CI - Corrected:', frac_free_dir[-1][1])
    print('Free vs Controls Dir Raw:', frac_free_dir[0])


    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/kruskal_frac_free_vis'] = pd.DataFrame(list(frac_free_vis), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
        store[f'{vis_class}vis/kruskal_frac_free_ori'] = pd.DataFrame(list(frac_free_ori), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
        store[f'{vis_class}vis/kruskal_frac_free_dir'] = pd.DataFrame(list(frac_free_dir), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T

In [ ]:
# frac_free_fig = plot_bootstrap_distributions([frac_free_gen, 
#                                               frac_free_vis, frac_free_vis_no_gen, 
#                                               frac_free_ori, frac_free_ori_no_gen,
#                                               frac_free_dir, frac_free_dir_no_gen])
# plt.show()

## Count Vis Tuning - Fixed vs Free

### Kruskal-Wallis

In [ ]:
# Do a Kruskal-Wallis test with posthoc Dunn test for multiple comparisons
for vis_class in vis_classes:

    # Visual
    count_exp_fixed_free_vis = kruskal_w_dunn(sample_df(main_exp_data['count_vis_resp_fixed'][f'{vis_class}visual'], 
                                                main_exp_data['count_vis_resp_free'][f'{vis_class}visual']))
    print('Fixed vs Free Visual:', exp_fixed_free_vis)

    # Orientation
    count_exp_fixed_free_ori = kruskal_w_dunn(sample_df(main_exp_data['count_vis_resp_fixed'][f'{vis_class}orientation'], 
                                                main_exp_data['count_vis_resp_free'][f'{vis_class}orientation']))
    print('Fixed vs Free Orientation:', exp_fixed_free_ori)


    # Direction
    count_exp_fixed_free_dir = kruskal_w_dunn(sample_df(main_exp_data['count_vis_resp_fixed'][f'{vis_class}direction'], 
                                                main_exp_data['count_vis_resp_free'][f'{vis_class}direction']))
    print('Fixed vs Free Direction:', exp_fixed_free_dir)

    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/kruskal_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
        store[f'{vis_class}vis/kruskal_exp_fixed_free_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
        store[f'{vis_class}vis/kruskal_exp_fixed_free_dir_count'] = pd.DataFrame(count_exp_fixed_free_dir, index=['p_val', 'p_val_dunn_posthoc'])

### Wilcoxon Rank Sum

In [ ]:
# Do a Wilcox Rank-Sum test with Bonferroni correction
for vis_class in vis_classes:

    # Visual
    count_exp_fixed_free_vis = wilcoxon_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'][f'{vis_class}visual'], 
                                                main_exp_data['count_vis_resp_free'][f'{vis_class}visual']))
    print('Fixed vs Free Visual:', exp_fixed_free_vis)

    # Orientation
    count_exp_fixed_free_ori = wilcoxon_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'][f'{vis_class}orientation'], 
                                                main_exp_data['count_vis_resp_free'][f'{vis_class}orientation']))
    print('Fixed vs Free Orientation:', exp_fixed_free_ori)

    # Direction
    count_exp_fixed_free_dir = wilcoxon_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'][f'{vis_class}direction'], 
                                                main_exp_data['count_vis_resp_free'][f'{vis_class}direction']))
    print('Fixed vs Free Direction:', exp_fixed_free_dir)

    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/wilcoxon_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
        store[f'{vis_class}vis/wilcoxon_exp_fixed_freew_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
        store[f'{vis_class}vis/wilcoxon_exp_fixed_free_dir_count'] = pd.DataFrame(count_exp_fixed_free_dir, index=['p_val', 'p_val_dunn_posthoc'])

### Mann-Whitney U

In [ ]:
# Do a Mann-Whitney U test with Bonferroni correction
for vis_class in vis_classes:

    # Visual
    count_exp_fixed_free_vis = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'][f'{vis_class}visual'], 
                                                main_exp_data['count_vis_resp_free'][f'{vis_class}visual']))
    print('Fixed vs Free Visual:', exp_fixed_free_vis)

    # Orientation
    count_exp_fixed_free_ori = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'][f'{vis_class}orientation'], 
                                                main_exp_data['count_vis_resp_free'][f'{vis_class}orientation']))
    print('Fixed vs Free Orientation:', exp_fixed_free_ori)

    # Direction
    count_exp_fixed_free_dir = mannwhitneyu_w_bonferroni(sample_df(main_exp_data['count_vis_resp_fixed'][f'{vis_class}direction'], 
                                                main_exp_data['count_vis_resp_free'][f'{vis_class}direction']))
    print('Fixed vs Free Direction:', exp_fixed_free_dir)

    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_vis_count'] = pd.DataFrame(count_exp_fixed_free_vis, index=['p_val', 'p_val_dunn_posthoc'])
        store[f'{vis_class}vis/mannwhitneyu_exp_fixed_freew_ori_count'] = pd.DataFrame(count_exp_fixed_free_ori, index=['p_val', 'p_val_dunn_posthoc'])
        store[f'{vis_class}vis/mannwhitneyu_exp_fixed_free_dir_count'] = pd.DataFrame(count_exp_fixed_free_dir, index=['p_val', 'p_val_dunn_posthoc'])

## Count Vis Tuning - Exp vs Controls

### Fixed Kruskal Wallis

In [ ]:
# Do a Kruskal-Wallis test with Dunn posthoc

for vis_class in vis_classes:

    # Visual
    count_fixed_vis = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'count_vis_resp_fixed', 'visual',   
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Fixed vs Controls Visual 95% CI:', count_fixed_vis[1][1])
    print('Fixed vs Controls Visual Raw:', count_fixed_vis[0])

    # Orientation
    count_fixed_ori = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'count_vis_resp_fixed', 'orientation',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Fixed vs Controls Orientation 95% CI:', count_fixed_ori[1][1])
    print('Fixed vs Controls Orientation Raw:', count_fixed_ori[0])

    # Direction
    count_fixed_dir = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'count_vis_resp_fixed', 'direction',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Fixed vs Controls Direction 95% CI:', count_fixed_dir[1][1])
    print('Fixed vs Controls Direction Raw:', count_fixed_dir[0])

    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/kruskal_count_fixed_vis'] = pd.DataFrame(list(count_fixed_vis), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
        store[f'{vis_class}vis/kruskal_count_fixed_ori'] = pd.DataFrame(list(count_fixed_ori), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
        store[f'{vis_class}vis/kruskal_count_fixed_dir'] = pd.DataFrame(list(count_fixed_dir), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T

### Free Kruskal Wallis

In [ ]:
# Do a Kruskal-Wallis test with Dunn posthoc
for vis_class in vis_classes:

    # Visual
    count_free_vis = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'count_vis_resp_free', 'visual',   
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Free vs Controls Visual 95% CI:', count_free_vis[1][1])
    print('Free vs Controls Visual Raw:', count_free_vis[0])

    # Orientation
    count_free_ori = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'count_vis_resp_free', 'orientation',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Free vs Controls Orientation 95% CI:', count_free_ori[1][1])
    print('Free vs Controls Orientation Raw:', count_free_ori[0])

    # Direction
    count_free_dir = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                    'count_vis_resp_free', 'direction',
                                    kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
    print('Free vs Controls Direction 95% CI:', count_free_dir[1][1])
    print('Free vs Controls Direction Raw:', count_free_dir[0])

    # Save the data
    with pd.HDFStore(output_path, 'a') as store:
        store[f'{vis_class}vis/kruskal_count_free_vis'] = pd.DataFrame(list(count_free_vis), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
        store[f'{vis_class}vis/kruskal_count_free_ori'] = pd.DataFrame(list(count_free_ori), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
        store[f'{vis_class}vis/kruskal_count_free_dir'] = pd.DataFrame(list(count_free_dir), columns=['pval', 'ci'],
                                                    index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T

# Running Modulated

## Fixed

In [ ]:
# Do a Kruskal-Wallis test with Dunn posthoc

# Running Modulated
frac_fixed_run_mod = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                      'frac_run_mod_fixed', 'run_modulated',
                                      kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Fixed vs Controls Run Mod. 95% CI', frac_fixed_run_mod[1][1])
print('Fixed vs Controls Run Mod. 95% CI - Corrected:', frac_fixed_run_mod[-1][1])
print('Fixed vs Controls Run Mod. Raw:', frac_fixed_run_mod[0])

# Running and Visually Modulated
frac_fixed_vis_run_mod = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                        'frac_run_mod_fixed', 'vis_run_modulated',
                                        kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Fixed vs Controls Vis. + Run Mod. 95% CI', frac_fixed_vis_run_mod[1][1])
print('Fixed vs Controls Vis. + Run Mod. 95% CI - Corrected:', frac_fixed_vis_run_mod[-1][1])
print('Fixed vs Controls Vis. + Run Mod. Raw:', frac_fixed_vis_run_mod[0])

# Save the data
with pd.HDFStore(output_path, 'a') as store:
    store[f'kruskal_frac_fixed_run_mod'] = pd.DataFrame(list(frac_fixed_run_mod), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store[f'kruskal_frac_fixed_vis_run_mod'] = pd.DataFrame(list(frac_fixed_vis_run_mod), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T

In [ ]:
# with pd.HDFStore(output_path, 'a') as store:
#     a = store['kruskal_frac_free_run_mod']
#     print(a.loc['ci', 'corrected_bootstrap_tests'])

## Free

In [ ]:
# Do a Kruskal-Wallis test with Dunn posthoc

# Running Modulated
frac_free_run_mod = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                      'frac_run_mod_free', 'run_modulated',
                                      kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Free vs Controls Run Mod. 95% CI', frac_free_run_mod[1][1])
print('Free vs Controls Run Mod. 95% CI - Corrected:', frac_free_run_mod[-1][1])
print('Free vs Controls Run Mod. Raw:', frac_free_run_mod[0])

# Running and Visually Modulated
frac_free_vis_run_mod = bootstrap_tests([main_exp_data, control_light_data, control_dark_data],
                                        'frac_run_mod_free', 'vis_run_modulated',
                                        kruskal_w_dunn, n_iters=bootstrap_iters, bootstrap_all=True)
print('Free vs Controls Vis. + Run Mod. 95% CI', frac_free_vis_run_mod[1][1])
print('Free vs Controls Vis. + Run Mod. 95% CI - Corrected:', frac_free_vis_run_mod[-1][1])
print('Free vs Controls Vis. + Run Mod. Raw:', frac_free_vis_run_mod[0])

# Save the data
with pd.HDFStore(output_path, 'a') as store:
    store[f'kruskal_frac_free_run_mod'] = pd.DataFrame(list(frac_free_run_mod), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T
    store[f'kruskal_frac_free_vis_run_mod'] = pd.DataFrame(list(frac_free_vis_run_mod), columns=['pval', 'ci'],
                                                  index=['raw_test', 'bootstrap_tests', 'corrected_bootstrap_tests']).T